#### Common preparations

In [ ]:
%pip install icrawler

In [ ]:
%pip install selenium

In [ ]:
%pip install webdriver_manager

In [ ]:
!apt install chromium-browser

In [ ]:
!apt install chromium-chromedriver

In [47]:
!chromium-browser --version

Chromium 85.0.4183.83 Built on Ubuntu , running on Ubuntu 18.04


In [34]:
# common libraries
import os
import pathlib
from timeit import default_timer as timer

from google.colab import drive

# For baidu, bing-crawler
from icrawler.builtin import BaiduImageCrawler, BingImageCrawler, GoogleImageCrawler

# for naver-crawler
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from urllib.parse import quote_plus

# for google-crawler
from urllib.request import urlretrieve
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [28]:
# Mount Google drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
# 작업 디렉토리로 이동(꼭, 꼭 확인하세요!)
%cd "/content/gdrive/My Drive/팀 프로젝트/crawling_data-google"
%pwd

/content/gdrive/.shortcut-targets-by-id/1J3XOAxwIL_8F7HmA8dtgu0h75Aj2Vvzx/팀 프로젝트/crawling_data-google


'/content/gdrive/.shortcut-targets-by-id/1J3XOAxwIL_8F7HmA8dtgu0h75Aj2Vvzx/팀 프로젝트/crawling_data-google'

In [25]:
# keyword preparation
keywords = {
               "grape": ["grape", "포도", "포도 한 송이", "葡萄", "ブドウ", ],
               "melon": ["yellow melon", "korean yellow melon", "참외", "マクワウリ", "甜瓜" ],
               "broccoli": ["broccoli", "브로콜리", "ブロッコリー", "花椰菜"],
               "carrot": ["carrot", "당근", "ニンジン", "胡萝卜"],
               "mandarin": ["mandarin orange", "귤", "감귤", "みかん", "柑桔"],
               "almond": ["almond", "아몬드", "アーモンド", "杏仁"],
               "onion": ["onion", "양파", "タマネギ", "洋葱"],
               "welchOnion": ["welch onion", "대파", "ネギ", "大葱"],
               "apple": ["red apple", "사과", "林檎", "リンゴ"],
}
# 96sec / kw

#### Google crawling 
- it needs longer codes than others

In [ ]:
driver = webdriver.Chrome('./팀 프로젝트/chromedriver.exe')
driver.get(url)

In [ ]:
keyword = "포도 한송이"
url = "https://www.google.com/search?q='keyword&sxsrf=ALeKk00Yi_ikiVVyl0uVCWdiyae1XFsTkA:1599009530790&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjI1MzYpsnrAhWCFIgKHSelBSIQ_AUoAXoECBkQAw&biw=1413&bih=803&dpr=1.13"

In [ ]:
for i in range(500):
    driver.execute_script('window.scrollBy(0,10000)')

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
img = soup.select('.rg_i.Q4LuWd')
n = 1 
imgurl = []

In [ ]:
for i in img:
    try:
        imgurl.append(i.attrs['src'])
    except KeyError:
        imgurl.append(i.attrs['data-src'])
        
for i in imgurl:
    urlretrieve(i, '건멸치/'+keyword+str(n)+'.jpg')
    n+=1
    print(imgurl)
    
    if(n==100):
        break
        
driver.close()

#### Naver crawling

In [ ]:
baseUrl = 'https://search.naver.com/search.naver?where=image&sm=tab_jum&query='
crawl_max_num = 50 # should be <= 50

for category in keywords:
  for keyword in keywords[category]:
    plusUrl = keyword
    url = baseUrl + quote_plus(plusUrl)
    html = urlopen(url)
    soup = bs(html, "html.parser")
    img = soup.find_all(class_='_img')

    n = 1
    for i in img:
      print(n, end=" ")
      imgUrl = i['data-source']
      with urlopen(imgUrl) as f:
        savedir = os.path.join('./', category, keyword)
        pathlib.Path(savedir).mkdir(parents=True, exist_ok=True)
        saveimg = os.path.join(savedir, plusUrl+"-"+str(n)+'.jpg')
        
        with open(saveimg,'wb') as h: # w - write b - binary
          img = f.read()
          h.write(img)
      n += 1
      if n > crawl_max_num:
        break


    print('\nImage crawling for keyword {} is done.'.format(keyword))
  print('\n\nImage crawling for category {} is done.'.format(category))

#### Baidu crawling

In [ ]:
start_time = timer()

for category in keywords:
    for keyword in keywords[category]:
      path = os.path.join('./', category, keyword)
      baidu_crawler = BaiduImageCrawler(downloader_threads=4, storage={'root_dir': path })

      baidu_crawler.crawl(keyword=keyword , offset=0, max_num=300, max_size=None)

end_time = timer()
print("time elapsed :", str((end_time - start_time)/60))

#### Bing crawling

In [ ]:
for category in keywords:
    for keyword in keywords[category]:
      path = os.path.join('./', category, keyword)
      bing_crawler = BingImageCrawler(downloader_threads=4, storage={'root_dir': path})

      bing_crawler.crawl(keyword=keyword , filters=None, offset=0, max_num=300)